In [ ]:
from pathlib import Path
import json
from dataclasses import dataclass
from IPython.display import display, clear_output
from PIL import Image


@dataclass(frozen=True)
class BBox:
    xmin: float
    ymin: float
    xmax: float
    ymax: float

    @classmethod
    def from_ltrb(cls, l, t, r, b):
        return cls(l, t, r, b)

    @classmethod
    def from_ltwh(cls, l, t, w, h):
        return cls(l, t, l + w, t + h)

    @classmethod
    def from_center_wh(cls, xc, yc, w, h):
        return cls(xc-w/2, yc-w/2, xc + w/2, yc + h/2)

    def __iter__(self):
        self_tuple = (self.xmin, self.ymin, self.xmax, self.ymax)
        yield from self_tuple

    @property
    def center(self):
        return (self.xmin + self.xmax) / 2, (self.ymin + self.ymax) / 2

    @property
    def size(self):
        return (- self.xmin + self.xmax), (-self.ymin + self.ymax)

    @property
    def as_center_wh(self):
        return (*self.center, *self.size)

    def scale(self, x: float = 1, y: float = 1):
        c = self.center
        w, h = self.size
        w *= x
        h *= y
        return BBox.from_center_wh(*c, w, h)

fold = "test"
source_dir = Path(f"./data/annotations-{fold}")
save_dir = Path(f"./data/watch-time-{fold}")
save_dir.mkdir(exist_ok=True)
img_dir = Path(f"./data/{fold}")
records = {}
asset_files = list(source_dir.glob("*.json"))
print(asset_files[0])
skipped = set()

In [ ]:
for i, file in enumerate(asset_files):
    with file.open('r') as f:
        data = json.load(f)
    image_name = data['asset']['name']
    regions = data['regions']
    watch_faces = [region for region in regions if 'watch-face' in region['tags']]
    for watch_face in watch_faces:
        region_id = watch_face['id']
        bbox = watch_face['boundingBox']
        l, t = bbox['left'], bbox['top']
        w, h = bbox['width'], bbox['height']
        box = BBox.from_ltwh(l,t,w,h)

        with Image.open(img_dir / image_name) as img:
            filename = f"{region_id}.jpg"
            crop = img.crop(tuple(map(int, box.scale(1.0,1.0))))
            time = None
            if (save_dir / filename).exists() or image_name in skipped:
                continue
            crop_copy = crop.copy()
            while time is None:
                display(crop_copy)
                decision = input()
                if decision=='skip':
                    time = -1
                    skipped.add(image_name)
                elif decision=='redo':
                    crop_copy = crop.copy()
                elif 'r' in decision:
                    angle = int(decision.split(':')[1])
                    crop_copy = crop_copy.rotate(angle)

                else:
                    time = decision.split(":")
                clear_output()
            if time != -1:
                
                records[filename] = {
                    'region_id': region_id,
                    'hour':int(time[0]),
                    'minute':int(time[1]),
                    "filename": filename
                }
                crop_copy.save(save_dir / filename)

In [ ]:
len(records)

In [ ]:
import pandas as pd
df = pd.DataFrame(list(records.values()))

In [ ]:
df

In [ ]:
df.to_csv(save_dir / 'labels.csv', index=False)

In [ ]:
df['minute'].hist(bins=60)

In [ ]:
df['hour'].hist(bins=12)